In [ ]:
import os
import numpy as np
from PIL import Image
import chainer
from chainer.datasets import LabeledImageDataset
from chainer.datasets import TransformDataset
import chainer.links as L
import chainer.functions as F
from chainer import serializers
from chainer import training
from chainer.training import extensions


# Network definition
class MLP(chainer.Chain):

    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(300, 256)  # 300(20*15) -> 256 units
            self.l2 = L.Linear(256, 256)  # 256 units -> 256 units
            self.l3 = L.Linear(256, 1)      # 256 units -> 1

    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


# Preprocess data
def preprocess(in_data):
    img, label = in_data
    img = img / 256.  # normalization
    img = img.reshape(300)  # 2-dim (64,48) -> 1-dim (3072)
    label = label.reshape(1)   # scalar -> 1-dim
    return img, label


input_dir = 'angle_data/20190830-170739'
out_dir = 'angle_model'
batchsize = 100
epoch = 200

# Set up a neural network to train
model = L.Classifier(MLP(),
                     lossfun=F.mean_squared_error,
                     accfun=F.mean_absolute_error)

# Setup an optimizer
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

# Load the dataset
dataset = LabeledImageDataset(os.path.join(input_dir, 'list.txt'),
                              os.path.join(input_dir, 'images'),
                              label_dtype=np.float32)

# Set a dataset
train = TransformDataset(dataset, preprocess)

# Set an iterator
train_iter = chainer.iterators.SerialIterator(train, batchsize)

# Setup an Updater
updater = training.updaters.StandardUpdater(train_iter, optimizer)

# Setup a Trainer
trainer = training.Trainer(updater, (epoch, 'epoch'), out='result')
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'elapsed_time']))

# Run the Trainer
trainer.run()

# Save the model, the optimizer and config.                                                                                                                         
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
print('save the model')
serializers.save_npz('{}/mychain.model'.format(out_dir), model.predictor)
print('save the optimizer')
serializers.save_npz('{}/mychain.state'.format(out_dir), optimizer)

